In 1998 Doug Gray wrote [a short paper](https://www.kilohotel.com/rv8/rvlinks/doug_gray/TAS_FNL4.pdf) describing a fantastic method for finding true airpseed vectors (airspeed and heading) from GPS vectors (groundspeed and ground track). Three GPS ground speed/track pairs in any 3 directions is about as easy as can be for a test flight.

Although his spreadsheet formulas work, I wanted to understand it more deeply and write some code. I re-learned some trig and geometry so I could re-derive what is going on here.

## Reference diagram
![Diagram](Diagram.png)

In [1]:
# preamble
import numpy as np
import numpy.linalg as la

def cartesian(rho, theta):
    x = rho * np.cos(theta)
    y = rho * np.sin(theta)
    return x, y

def polar(v):
    x, y = v
    rho = np.hypot(x, y)
    theta = np.arctan2(y, x)
    return rho, theta
    
def fromCompass(speed, track):
    theta = np.radians(90 - track)
    return cartesian(speed, theta)

def toCompass(v):
    speed, theta = polar(v)
    track = (90 - np.degrees(theta)) % 360
    return speed, track

def mag(v):
    return np.sqrt(v[0]**2 + v[1]**2)
    
def angleBetween(u, v):
    return np.arccos(np.dot(u, v) / (mag(u) * mag(v)))

def angle(v):
    x, y = v
    return np.arctan2(y, x)

In [2]:
# flight test data (groundspeed, ground track in degrees)
data = [
        (140, 192),
        (112, 283),
        (120, 20)
]
g1, g2, g3 = (fromCompass(speed, track) for speed, track in data)

In [3]:
def calculate(data):
        g1, g2, g3 = (fromCompass(speed, track) for speed, track in data)
        p1 = np.subtract(g1, g2)
        p2 = np.subtract(g1, g3)
        m1 = -p1[0]/p1[1]
        m2 = -p2[0]/p2[1]
        b1 = (p1[1] - m1 * p1[0]) / 2
        b2 = (p2[1] - m2 * p2[0]) / 2
        x = (b2-b1)/(m1-m2)
        a1 = [x, m1 * x + b1]
        w = np.subtract(g1, a1)
        a2 = np.subtract(g2, w)
        a3 = np.subtract(g3, w)
        tas = mag(a1)

        return tas, w, a1, a2, a3

Doug's results:
- TAS: 130
- Headings: 200°, 287.8°, 11.7°
- Wind: (20.6, 314.8°)


In [4]:
tas, w, a1, a2, a3 = calculate(data)
print("\nOur results:\n")
print(f"  TAS: {tas}")
print(f"  Headings: {[theta for rho, theta in (toCompass(v) for v in (a1, a2, a3))]}")
# Wind is stated by where it is coming *from*
print(f"  Wind: {toCompass(-w)}")


Our results:

  TAS: 129.99852349653085
  Headings: [199.67059374146783, 287.792125444477, 11.713025864830456]
  Wind: (20.63344402295658, 314.7584466666683)


## References
- [Using GPS to accurately establish True Airspeed (TAS)](https://www.kilohotel.com/rv8/rvlinks/doug_gray/TAS_FNL4.pdf), Doug Gray, 1998
- [Flight Testing: Finding TAS from GPS Data](https://www.kitplanes.com/flight-testing-finding-tas-from-gps-data/), Kevin Horton, 2009

TODO
- update diagram with new names
- explicate